In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [15]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
# image = load_image("example_data/corgi.png")
# image = load_image("example_data/cube.png")
image = load_image("../../../content/cube_tall.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:

# load latents from file
latents_load = torch.load('../../latents/cube_tall_latents.pt')

In [16]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
# size = 64 # this is the size of the renders; higher values take longer to render.
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))
    # save image to disk
    images[0].save('../../../content/cube_tall_{}.gif'.format(i), save_all=True, append_images=images[1:], duration=100, loop=0)

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOW7O+S7PeS7POS7O+W7OuS7OuW6OuS6POS6O+S6OuO7P+O6P+…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAObCLuXCMOXCLubBLuXBMOXBL+XBLubBLOXBLeXALuXALeXALO…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOW9Q+W9Qua9QeW9QeW8ROW8Q+W8QuW8QeS8RuS8ReS8ROO8Ru…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOS4PuS4PeO4P+O4PuO4PeS3PeO3P+O3PuO3PeK4P+K3P+K3Pu…

In [17]:
# save latents to file
torch.save(latents, '../../latents/cube_tall_latents.pt')